# E-commerce data - Regressione

Il dataset `E-commerce` (disponibile su [Kaggle](https://www.kaggle.com/datasets/carrie1/ecommerce-data/data)) contiene informazioni sui clienti di un negozio di e-commerce che vende vestiti. Il marchio offre sessioni di styling e consigli di moda con un personal stylist all'interno del negozio. I clienti possono poi ordinare gli abiti desiderati da casa utilizzando l'app mobile o il sito web.

L'azienda sta decidendo se investire per migliorare l'esperienza sull'app o sul sito web. L'obiettivo è cercare di ricavare dai dati alcune informazioni utili per prendere questa decisione.

In questo notebook utilizzeremo un dataset che deriva dal dataset appena descritto al quale sono state apportate alcune modifiche. Ogni riga del dataset descrive le caratteristiche di un cliente. Tra le variabili del dataset è presente anche la spesa annua totale effettuata da tale cliente. L'obiettivo sarà quello di sviluppare un modello di **regressione**, che ci permetterà di prevedere la spesa totale e di analizzarne i coefficienti per capire quali siano i fattori più determinanti.

Nella prima parte analizzeremo i dati e li prepareremo per la parte di implementazione del modello, nella seconda parte implementeremo un modello di regressione lineare semplice e faremo un confronto con altri modelli di regressione.

**Nota**: scorrendo il notebook noterai che ci sono alcune celle di codice che contengono puntini (...), quelle sono le parti che dovrai completare aiutandoti con le esercitazioni che abbiamo svolto durante le lezioni e con le presentazioni che avete seguito. Altre celle sono invece già completate e dovrai solo cliccare 'run' per studiarne l'output. Troverai anche alcune domande che ti guideranno nella descrizione e nell'analisi che svolgerai per il report finale.

<a id="0"></a> <br>

# Indice
1. [Pre-processing dei dati](#1)
2. [Exploratory Data Analysis](#2)
3. [Implementazione e valutazione del modello di Regressione Lineare](#3)
4. [Altri modelli di regressione](#4)

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

<a id="1"></a> <br>
## 1. Pre-processing dei dati


Lo step fondamentale prima di applicare un modello di Machine Learning è quello di studiare le caratteristiche principali dei dati per renderli utilizzabili dal modello che sceglieremo. In questa sezione, metteremo in pratica le tecniche viste nelle precedenti lezioni per analizzare il dataset, in particolare dovremo:
- caricare il dataset
- estrarre le prime descrizioni generali (dimensione, tipo di dati, variabili, ...)
- gestire i valori mancanti

- caricamento del dataset e descrizione generale

In [ ]:
# Caricamento del dataset 'Ecommerce_Customers_mod.csv'
df = pd.read_csv('../data/Ecommerce_Customers_mod.csv')

# Stampa le prime dieci righe
...

In [ ]:
# Che dimensione ha il dataset? Quante righe e quante colonne ha?
...

In [ ]:
# Quali sono i nomi delle variabili presenti nel dataset? I nomi delle variabili sono contenuti nella lista delle colonne del dataframe
...

Le variabili incluse nel dataset sono:

- **Email**: indirizzo e-mail
- **Indirizzo**: indirizzo di consegna
- **Avatar**: nickname
- **Avg. Session Length**: tempo medio trascorso nelle sessioni di consulenza stilistica in negozio (in minuti)
- **Time on App**: tempo medio trascorso sull'app (in minuti)
- **Time on Website**: tempo medio trascorso sul sito web (in minuti)
- **Length of Membership**: da quanti anni il cliente è membro
- **Yearly Amount Spent**: importo totale speso durante un anno
- **Age**: età del cliente
- **City District**: quartiere della città in cui il cliente vive

In [ ]:
# Alcune informazioni importanti sul dataset
...

In [ ]:
# Tipo di dato in ogni colonna
...

In [ ]:
# Caratteristiche statistiche principali per le variabili numeriche
...

In [ ]:
# Controlla se ci sono valori duplicati e in caso affermativo rimuovili
# Numero di righe duplicate
...

In [ ]:
# Righe duplicate
df[df.duplicated(keep=False)]

In [ ]:
# Rimuovi le righe duplicate
...

Quale potrebbe essere una motivazione per queste righe dupicate?

- valori mancanti

In [ ]:
# Quanti valori nulli ci sono in ogni colonna?
...

In [ ]:
# Controlla quali righe contengono valori nulli
df[df.isna().any(axis=1)]

In [ ]:
# Elimina le righe che hanno almeno un valore mancante
...

In [ ]:
# Controlla che l'operazione sia andata a buon fine e non siano rimasti valori mancanti
...

Perchè in questo caso può essere sensato eliminare direttamente le righe con valori mancanti? Quale potrebbe essere un'alternativa?

In [ ]:
# Resettiamo il valore degli indici per evitare problemi successivi
df.reset_index(inplace=True, drop=True)

- variabili categoriche

Ci sono variabili categoriche nel dataset?

Pensi che alcune di queste possano essere importante per prevedere la spesa totale di un cliente? Come possiamo trasformarle in varaibili numeriche?

Abbiamo visto due metodi per trasformare le variabili categoriche in numeriche: OrdinalENcoder e OneHotEncoder. Quali sono le loro caratteristiche? In quali contesti possono essere più adatti? Quali problematiche vedi nella loro applicazione in questo problema?

In [ ]:
# Quali valori unici assume la variabile "Seasonal Shopping Preference"?
...

In [ ]:
# One-Hot Encoding per 'Seasonal Shopping Preference'
from sklearn.preprocessing import OneHotEncoder

# Inizializzazione dell'encoder
encoder = OneHotEncoder(sparse_output=False)

# Slezioniamo la variabile categorica da codificare
genre_encoded = encoder.fit_transform(df[['Seasonal Shopping Preference']])

# Convertiamo i dati codificati in un DataFrame con nomi delle colonne significativi
genre_encoded_df = pd.DataFrame(genre_encoded, columns=encoder.get_feature_names_out(['Seasonal Shopping Preference']))

# Aggiungiamo le colonne codificate nel DataFrame originale
df = pd.concat([df, genre_encoded_df], axis=1)

# Eliminiamo la colonna originale 'Embarked' dal dataset
df = df.drop(['Seasonal Shopping Preference'], axis=1)

# Mostriamo il DataFrame aggiornato
df.head()

<a id="2"></a> <br>

## 2. Exploratory data analysis (EDA)

In questa sezione utilizzeremo dei metodi di visualizzazione dei dati per continuare ad analizzare le caratteristiche del dataset. 
In particolare dovremo:
- plottare la correlation heatmap per valutare la correlazione tra le diverse variabili
- distribuzione delle variabili

- correlazione

In [ ]:
# Stampiamo la correlation heatmap per valutare la correlazione tra le variabili
plt.figure(figsize=(10, 8))
sns.heatmap(..., annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap')
plt.show()

Quali sono le variabili che hanno la correlazione più alta? Sapresti spiegare il perchè?

- distribuzioni

In [ ]:
# Studiamo la distribuzione delle variabili presenti nel dataset
...(bins=25,figsize=(20,10))
plt.show()

In [ ]:
# Dividiamo i clienti in tre gruppi in base all'età. Qual è il gruppo che ha più clienti?
bins = [0, 35, 50, 100]
labels = ['0-35', '35-50', '50-100']

plt.figure(figsize=(4,5))
sns.histplot(pd.cut(df['Age'], bins=bins, labels=labels, right=False), kde=False, discrete=True, color='skyblue')
plt.title('Age Distribution by Group')
plt.xlabel('Age Group')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Come cambiano le spese annuali totali dei clienti a seconda del gruppo?
plt.figure(figsize=(5,4))
plt.title("Yearly Amount Spent by Age Group")
plt.xlabel("Age Group")
plt.ylabel("Yearly Amount Spent")
plt.xticks(rotation=45)

df['Age Group'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False)
grouped = df.groupby('Age Group')['Yearly Amount Spent'].mean()
grouped.plot(kind='bar', color='skyblue')

plt.show()

Cosa puoi dedurre da questo grafico? La variabile 'Age' sarà importante per prevedere la spesa totale?

<a id="3"></a> <br>

## 3. Implementazione e valutazione del modello di Regressione Lineare

In questa sezione costruiremo e alleneremo il modello di regressione lineare per cercare di prevedere il numero totale di casi per milione di abitanti (seguendo gli step illustrati nella presentazione). Infine valuteremo il modello ottenuto calcolando diverse metriche.

LinearRegression (documentazione): https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error , mean_absolute_percentage_error , mean_squared_error

In [ ]:
df.columns

In [ ]:
# Quali pensi possano essere le variabili utili per la regressione? 
# Seleziona una lista di variabili che utilizzerai per costruire il modello di regressione
# N.B.: se vuoi a inserire tra le variabili anche quelle relative alla preferenza della stagione, fai attenzione a inserirne solo due delle tre 
# per evitare problemi nel modello dovuti a lla multicollineraità. Infatti quando si usa il metodo do One Hot ENcoding per 
# decodificare una variabile categorica che ha N categoria diverse bisogna ricordarsi di includerne solo N-1 nel modello.
# Questo perchè se dessimo tutte le N nuove colonne, staremmo dando troppe informazioni al modello (infatti avendo le N-1 colonne 
# è possibile ricavarsi quella mancante) e lo porteremmo a fare errori nelle previsioni o a trovare coefficienti per le variabili molto grandi e instabili
features = [...]

In [ ]:
# Definiamo le variabili di input (X) e di output (y)
# Il nome della variabile da prevedere è la spesa totale annua (Yearly Amount Spent)
X = df[features]
y= df['...']

In [ ]:
# Dividiamo i dati in training (80%) e test(20%)
...

In [ ]:
# Controlliamo la dimensione del dataset di training e di test
...
...
...
...

In [ ]:
# Standardizziamo i dati
ro_scaler=StandardScaler()
X_train=ro_scaler.fit_transform(X_train)
X_test=ro_scaler.transform(X_test)

In [ ]:
# Creiamo il modello di Regressione Lineare
model = ...

In [ ]:
# Alleniamo il modello sui dati di training
...

In [ ]:
# Facciamo previsioni sui dati di test
y_predict = ...


In [ ]:
# Valutiamo il modello
# calcoliamo le diverse metriche (R2, Mean Squared Error (MSE), Root Mean Squared Error (RMSE), Mean Absolute Error (MAE), Mean Absolute Percentage Error (MAPE))
r_squared = r2_score(... , ...)
mse = mean_squared_error(..., ...)
rmse = np.sqrt(mse)
mae = mean_absolute_error(...,...)
mape = mean_absolute_percentage_error(... , ...) # Misura l'errore medio in percentuale rispetto ai valori reali

print('R squared (R2):',r_squared)
print(f'Mean Squared Error (MSE):{mse}')
print(f'Root Mean Squared Error (RMSE):{rmse}')
print(f'Mean Absolute Error (MAE):{mae}')
print("Mean  absolute precentage error of linear regression : ",mape*100,'%')

In [ ]:
# Coefficienti del modello
model.coef_

In [ ]:
# Creaiamo un piccolo dataFrame che associ ad ogni variabile utilizzata nel modello il coefficiente trovato
# (se il coeff)
threshold = 1e6  # Only round numbers smaller than 1 million
cdf = pd.DataFrame(np.where(np.abs(model.coef_) < threshold, np.round(model.coef_, 2), 0),X.columns,columns=['Coef'])
print(cdf)

Cosa rappresentano i coefficienti della linea di regressione multipla? Quali sono le variabili che hanno un coefficiente più alto? Cosa puoi dedurre?

Facoltativo: puoi provare altri modelli di regressione tra quelli visti nell'esercitazione a lezione